In [ ]:
from pathlib import Path
from traitlets import All
from ipyvuetify import Container, Btn, Row, TextField, Divider, Form, ProgressLinear, Icon, Spacer
from ipyvuetify import FileInput
from ipywidgets import Output, jslink
from IPython.display import Markdown

from molino import prepare_transactions_db
from molino.importers import alf, pml
from molino.transactions import transactions_db, Classification


In [ ]:
def import_trace_in_db(trace: str | Path, database: str | Path):
    """Import observation trace in database."""
    # TODO Test if trace has already been processed in DB (detect if import complete or partial?)
    # TODO If DB exists should be sqlite DB
    # TODO Infer trace type from extension (observation or pml)
    # TODO Advance progress from this function
    # TODO Remove chunking from ALF importer and perform it here
    if isinstance(trace, str):
        pattern = Path(trace)
        traces = list(pattern.parent.glob(pattern.name))
    elif isinstance(database, Path):
        traces = [trace]
    else:
        raise ValueError(f"Unsupported trace type '{type(trace)}'")
    database = Path(database)
    if len(traces) == 0:
        raise FileNotFoundError(f"No trace matching '{trace}' found")
    if any(not t.exists() for t in traces):
        raise FileNotFoundError(f"File {trace} does not exist")
    if database.exists() and not database.is_file():
        raise FileExistsError(f"{database} already exists but is not a file")
    prepare_transactions_db(str(database))
    alf.load_observations(*traces)

def import_pml_in_db(trace: str | Path, database: str | Path):
    """Import PML classification trace in database."""
    if isinstance(trace, str):
        pattern = Path(trace)
        traces = list(pattern.parent.glob(pattern.name))
    elif isinstance(database, Path):
        traces = [trace]
    else:
        raise ValueError(f"Unsupported trace type '{type(trace)}'")
    database = Path(database)
    if len(traces) == 0:
        raise FileNotFoundError(f"No trace matching '{trace}' found")
    if any(not t.exists() for t in traces):
        raise FileNotFoundError(f"File {traces} do not exist")
    if database.exists() and not database.is_file():
        raise FileExistsError(f"{database} already exists but is not a file")
    prepare_transactions_db(str(database))
    pml.load_pml_classifications(*traces, create_missing=False, batch_size=2048)


# Import observations

The following form lets you import an observation trace, in the `.alf` file format, into a Molino database.
Provide the path to a database file, the path to a trace file, and click the import button.
The importer does support `bz2`-compressed traces. The database will be created if required.
Note that in binder, a sample trace is accessible at : `../../../samples/riscv/obs/fu740_c1_batch0-no_locking.txt.bz2`.


In [ ]:
obs_console = Output()

obs_db_fs = TextField(
    clearable=True,
    label="Path to database file",
    prepend_icon="mdi-database",
    v_model="",
)

observation_trace_fs = TextField(
    clearable=True,
    label="Path to trace file",
    prepend_icon="mdi-file-outline",
    v_model="",
)

obs_import_btn = Btn(
    children=["Import"]
)

def do_obs_import(widget, event, data):
    obs_console.clear_output()
    with obs_console:
        obs_import_btn.loading = True
        try:
            import_trace_in_db(observation_trace_fs.v_model, obs_db_fs.v_model)
        finally:
            obs_import_btn.loading = False

obs_import_btn.on_event("click", do_obs_import)

obs_import_form = Form(
    children=[
        Container(
            children=[
                Row(children=[obs_db_fs]),
                Row(children=[observation_trace_fs]),
                Row(children=[obs_import_btn]),
                Row(children=[obs_console]),
            ]
        ),
    ]
)


obs_import_form



# Import PML classifications

The following form lets you import a PML classification trace, as a `.txt` file, into a Molino database.
Provide the path to the database file, the path to the classification file, and click the import button.
The database will be created if required.

In [ ]:
pml_console = Output()

pml_db_fs = TextField(
    clearable=True,
    label="Path to database file",
    prepend_icon="mdi-database",
    v_model="",
)

pml_trace_fs = TextField(
    clearable=True,
    label="Path to trace file",
    prepend_icon="mdi-file-outline",
    v_model="",
)

pml_import_btn = Btn(
    children=["Import"]
)

pml_purge_btn = Btn(
    children=[
        Icon(
            dark=True,
            children="mdi-skull",
        ),
        "Purge classifications",
    ]
)


def do_pml_import(widget, event, data):
    pml_console.clear_output()
    with pml_console :
        pml_import_btn.loading = True
        try:
            import_pml_in_db(pml_trace_fs.v_model, pml_db_fs.v_model)
        finally:
            pml_import_btn.loading = False

def do_purge_classifications(widget, event, data):
    pml_console.clear_output()
    with pml_console :
        widget.loading = True
        try:
            database = Path(pml_db_fs.v_model)
            if database.exists() and not database.is_file():
                raise FileExistsError(f"{database} already exists but is not a file")
            prepare_transactions_db(str(database))
            if transactions_db.is_connection_usable():
                q = Classification.delete()
                q.execute()
        finally:
            widget.loading = False

pml_import_btn.on_event("click", do_pml_import)
pml_purge_btn.on_event("click", do_purge_classifications)

pml_import_form = Form(
    children=[
        Container(
            children=[
                Row(children=[pml_db_fs]),
                Row(children=[pml_trace_fs]),
                Row(children=[pml_import_btn, Spacer(), pml_purge_btn]),
                Row(children=[pml_console]),
            ]
        ),
    ]
)

pml_import_form